In [1]:
ljy_fp = r"C:\Users\13377\Desktop\hw-memory-master (1)\hw-memory-master\benchmarks\objdump-D\-dmanytest.asm"
GuGuJi_fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/hw-memory/benchmarks/newlbm.asm"
GuGuJi_oldfp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/hw-memory/benchmarks/old_benchmark/spec_example/spec2006_470.lbm/lbm_part.asm"
fp = ljy_fp

In [2]:
'''读指令, 指令对象化'''
from newCFG.cfg import proc_identify
from newCFG.isa import Instruction, AddrMode
from typing import Tuple
from newCFG.read_asm import AsmFileReader, StatementType

reader =AsmFileReader(fp)

statements = list()
temp_stat = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
        temp_stat.append(Instruction(s[1]))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

        

# for i in temp_stat:
#     if i.is_ls:
#         if i.ls_handle:
#             print(i.tokens,i.ls_reg_target,i.ls_addr_mode,i.ls_addr_offset,i.local_offset)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
print()

_init 0000000000400498
.plt 00000000004004b0
__libc_start_main@plt 00000000004004d0
__gmon_start__@plt 00000000004004e0
abort@plt 00000000004004f0
_start 0000000000400500
__wrap_main 0000000000400534
_dl_relocate_static_pie 0000000000400540
call_weak_fn 0000000000400544
deregister_tm_clones 0000000000400560
register_tm_clones 0000000000400590
__do_global_dtors_aux 00000000004005d0
frame_dummy 0000000000400600
main 0000000000400604
_fini 0000000000400680



In [3]:
'''分proc'''
from graphviz import Digraph
from newCFG.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output\\procedures.svg'

In [4]:
'''处理call_graph'''
from newCFG.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output\\call_graph.svg'

In [5]:
'''生成TCFG和loop信息'''
from newCFG.cfg import TCfg
import random

tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()
tcfg.build_loop_hrchy()



#for n in tcfg_nodes:
#    print(n.name,"n的值",n.node_value)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)




In [6]:
'''设置node和edge上的的value'''

for l in tcfg.loops:
    l.back_edge.is_backEdge = True
for n in tcfg_nodes:
    n.set_rw_data()

temp_bool = True

while temp_bool:
    no_end = False

    for n in tcfg_nodes:
        n.set_rw_value()
        if not n.no_out:
            no_end = True
        temp_bool = no_end

for e in tcfg_edges:
    if e.is_backEdge:
        e.loop_value = random.randint(0,10)
        e.edge_value = 2

for e in tcfg_edges:
    print(e.src.name,e.dst.name,e.edge_value)

In [7]:
'''ls分析和读写特征'''

from newCFG.cfg import TCfg
from newCFG.loadstore import LSProc
from newCFG.rw_condition import RWProc, RWType

lsproc = LSProc(tcfg_nodes)

lds_table = lsproc.ls_table

for i in lds_table:
    i.final_addr
    i.local_offset

for i in lds_table:
    print(i.ins.tokens,i.ins.final_addr)

rwproc = RWProc(lds_table)

Global_Tolerant_value = 0
Global_Intolerant_value = 0

for rwu in rwproc.rw_table:
    #print(rwu.ins.tokens,rwu.find_cycle,rwu.ins.final_addr,rwu.is_torrent)
    if rwu.is_torrent == RWType.Global_Tolerant:
        Global_Tolerant_value += rwu.find_cycle
    elif rwu.is_torrent == RWType.Global_Intolerant:
        Global_Intolerant_value += rwu.find_cycle
        print(rwu.ins.tokens)

print("全局的容错路径为：",Global_Tolerant_value)
print("全局的非容错路径为：",Global_Intolerant_value)   

('40060c', 'b9400000', 'ldr', None, 'w0, [x0]') 4325420
('40061c', 'b9000001', 'str', None, 'w1, [x0]') 4325420
('400628', 'bd400000', 'ldr', None, 's0, [x0]') 4325416
('40064c', 'bd000000', 'str', None, 's0, [x0]') 4325416
('400658', 'b9400001', 'ldr', None, 'w1, [x0]') 4325420
('400664', 'b9400000', 'ldr', None, 'w0, [x0]') 4325424
('400674', 'b9000001', 'str', None, 'w1, [x0]') 4325488
('400658', 'b9400001', 'ldr', None, 'w1, [x0]')
全局的容错路径为： 13
全局的非容错路径为： 15


In [8]:
'''cache分析'''

#打印给cache分许需要的数据
for i in lds_table:
    print("指令",i.ins.tokens,"是否是SP",i.is_sp,"指令地址",i.ins.addr.val(),"访存地址",i.final_addr,"数据宽度",i.ins.ls_data_width,i.ins.ls_addr_mode)


指令 ('40060c', 'b9400000', 'ldr', None, 'w0, [x0]') 是否是SP False 指令地址 4195852 访存地址 4325420 数据宽度 32 AddrMode.Base
指令 ('40061c', 'b9000001', 'str', None, 'w1, [x0]') 是否是SP False 指令地址 4195868 访存地址 4325420 数据宽度 32 AddrMode.Base
指令 ('400628', 'bd400000', 'ldr', None, 's0, [x0]') 是否是SP False 指令地址 4195880 访存地址 4325416 数据宽度 32 AddrMode.Base
指令 ('40064c', 'bd000000', 'str', None, 's0, [x0]') 是否是SP False 指令地址 4195916 访存地址 4325416 数据宽度 32 AddrMode.Base
指令 ('400658', 'b9400001', 'ldr', None, 'w1, [x0]') 是否是SP False 指令地址 4195928 访存地址 4325420 数据宽度 32 AddrMode.Base
指令 ('400664', 'b9400000', 'ldr', None, 'w0, [x0]') 是否是SP False 指令地址 4195940 访存地址 4325424 数据宽度 32 AddrMode.Base
指令 ('400674', 'b9000001', 'str', None, 'w1, [x0]') 是否是SP False 指令地址 4195956 访存地址 4325488 数据宽度 32 AddrMode.Base


In [9]:
from cache_analysis.read_segment import segmentReader
from newCFG.heat_analysis import loop_heat
bread = segmentReader(r'C:\Users\13377\Desktop\hw-memory-master (1)\hw-memory-master\benchmarks\objdump-D\-D manytest.asm')
test=loop_heat(tcfg,lsproc,bread)
test.loop_heat_analysis()



n0 None
n0
load {(4325420, 4325424): 2, (4325416, 4325420): 1, (4325424, 4325428): 1}
store {(4325420, 4325424): 1, (4325416, 4325420): 1, (4325488, 4325492): 1}
